In [ ]:
import uproot
import pandas as pd

In [ ]:
uproot.open("/eos/user/k/kvadla/OpenData/2lep/Data/data16.periodA-D.root")["mini"].keys()
#uproot.open("/eos/user/k/kvadla/OpenData/2lep/MC/BSM_Signal_Samples/mc15.MGPy8EG.C1N2_WZ_2L2J.root")["mini"].keys()

In [ ]:
l_keys = uproot.open("/eos/user/k/kvadla/OpenData/2lep/Data/data16.periodA-D.root")["mini"].keys()
#l_keys = uproot.open("/eos/user/k/kvadla/OpenData/2lep/MC/BSM_Signal_Samples/mc15.MGPy8EG.C1N2_WZ_2L2J.root")["mini"].keys()
len(l_keys)

In [ ]:
tree = uproot.open("/eos/user/k/kvadla/OpenData/2lep/Data/data16.periodA-D.root")["mini"]
#tree = uproot.open("/eos/user/k/kvadla/OpenData/2lep/MC/BSM_Signal_Samples/mc15.MGPy8EG.C1N2_WZ_2L2J.root")["mini"]
tree

In [ ]:
df_flat = tree.pandas.df(entrystart=None, entrystop=100, flatten=None)
df_flat

In [ ]:
l_cuts_presel = ["trigE | trigM", "lep_n >= 2", "jet_n >= 2"]
df_flat = df_flat.query("&".join(l_cuts_presel))
df_flat

In [ ]:
tree.pandas.df(branches="jet_[!t]*", entrystart=None, entrystop=100, flatten=True)

In [ ]:
df_jet = tree.pandas.df(branches="/jet_(?!tru).*/i", entrystart=None, entrystop=100, flatten=True)
#df_jet = tree.pandas.df(branches="jet*", entrystart=None, entrystop=100, flatten=True)
df_jet = df_jet.loc[:,"jet_pt":]
#df_jet.unstack()
df_jet

In [ ]:
l_cuts_jet = ["jet_pt > 30e3", "abs(jet_eta) < 2.8"] 
l_cuts_bjet = ["jet_MV2c10 < 0.6459"] # 77% WP: 0.6459; 85% WP: 0.1758
df_jet = df_jet.astype({"jet_eta":float})
df_jet = df_jet.query("&".join(l_cuts_jet + l_cuts_bjet))
df_jet

In [ ]:
n_jet_sig = df_jet.groupby(level=0).size().values
df_jet = df_jet.unstack()
df_jet = df_jet[n_jet_sig == 2]
df_jet

In [ ]:
df_jet = df_jet.stack()
df_jet

In [ ]:
df_jet["subindex"] = df_jet.groupby(level=0).apply(lambda x : x.sort_values("jet_pt", ascending=False)).groupby(level=0).cumcount()
df_jet["subindex"]

In [ ]:
df_jet = df_jet.set_index("subindex", append=True)
df_jet

In [ ]:
df_jet = df_jet.reset_index(level="subentry").drop("subentry", axis=1)
df_jet

In [ ]:
df_jet = df_jet.unstack(level="subindex")
df_jet

In [ ]:
df_jet.columns = [ col[0] + str(col[1]+1) for col in df_jet.columns.values]
df_jet.columns

In [ ]:
df_jet

In [ ]:
from ROOT import TLorentzVector
def invariantMass(l_4vector1, l_4vector2):
    tlv1 = TLorentzVector()
    tlv2 = TLorentzVector()
    tlv1.SetPtEtaPhiE(l_4vector1[0], l_4vector1[1], l_4vector1[2], l_4vector1[3])
    tlv2.SetPtEtaPhiE(l_4vector2[0], l_4vector2[1], l_4vector2[2], l_4vector2[3])
    return (tlv1+tlv2).M()

In [ ]:
df_jet['mjj'] = df_jet.apply(lambda x: invariantMass([x.jet_pt1, x.jet_eta1, x.jet_phi1, x.jet_E1], [x.jet_pt2, x.jet_eta2, x.jet_phi2, x.jet_E2]), axis=1)
df_jet

In [ ]:
df_flat = pd.concat([df_flat, df_jet], axis=1, sort=False)
df_flat

In [ ]:
df_lep = tree.pandas.df(branches="/lep_(?!tru).*/i", entrystart=None, entrystop=100, flatten=True)
df_lep = df_lep.loc[:,"lep_pt":]
df_lep

In [ ]:
l_cuts_lep = ["lep_pt > 25e3",
              "((lep_type == 11) & (abs(lep_eta) < 2.47)) | ((lep_type == 13) & (abs(lep_eta) < 2.4))"]
df_lep = df_lep.query(" & ".join(l_cuts_lep))
df_lep

In [ ]:
df_lep = df_lep.unstack()
df_lep

In [ ]:
df_lep = df_lep[n_lep_sig == 2]
df_lep

In [ ]:
df_lep = df_lep.stack()
df_lep

In [ ]:
df_lep_sorted = df_lep.groupby(level=0).apply(lambda x : x.sort_values("lep_pt", ascending=False))
df_lep_sorted = df_lep_sorted.reset_index(level=0).drop("entry", axis=1)
df_lep["subindex"] = df_lep_sorted.groupby(level=0).cumcount()
df_lep = df_lep.set_index("subindex", append=True)
df_lep = df_lep.reset_index(level="subentry").drop("subentry", axis=1)
df_lep = df_lep.unstack(level="subindex")
df_lep

In [ ]:
df_lep.columns = [col[0]+str(col[1]+1) for col in df_lep.columns.values]
df_lep.columns

In [ ]:
df_lep

In [ ]:
l_cuts_SFOS = ["lep_type1 == lep_type2", "lep_charge1 != lep_charge2"]
df_lep = df_lep.query("&".join(l_cuts_SFOS))
df_lep

In [ ]:
df_flat = pd.concat([df_flat, df_lep], axis=1, sort=False)
df_flat

In [ ]:
df_flat.dropna(axis='index', inplace=True)
df_flat

In [ ]:
df_flat.values[0]*1

In [ ]:
l_features = ["lep_pt1", "lep_eta1", "lep_phi1", "lep_E1",
              "lep_pt2", "lep_eta2", "lep_phi2", "lep_E2",
              "jet_pt1", "jet_eta1", "jet_phi1", "jet_E1",
              "jet_pt2", "jet_eta2", "jet_phi2", "jet_E2",
              "met_et", "met_phi"]
df_flat_feat = df_flat[l_features]
df_flat_feat

In [ ]:
l_eventweights = ['mcWeight',
                  'scaleFactor_PILEUP',
                  'scaleFactor_ELE',
                  'scaleFactor_MUON',
                  'scaleFactor_BTAG',
                  'scaleFactor_LepTRIGGER']
df_flat[l_eventweights]

In [ ]:
s_eventweights = df_flat.scaleFactor_PILEUP * df_flat.scaleFactor_ELE * df_flat.scaleFactor_MUON * df_flat.scaleFactor_BTAG * df_flat.scaleFactor_LepTRIGGER
s_eventweights

In [ ]:
import numpy as np

In [ ]:
s_ew = pd.Series(1, index=np.arange(len(df_flat)))
s_ew

In [ ]:
df_flat[l_eventweights].agg('prod', axis='columns')

In [ ]:
from sklearn.utils import shuffle
df_flat = shuffle(df_flat)
df_flat

In [ ]:
ndarr_feat = df_flat.values*1
ndarr_feat

In [ ]:
ndarr_feat.shape

In [ ]:
len(ndarr_feat)

In [ ]:
y_zeros = np.zeros(len(ndarr_feat))
y_ones = np.ones(len(ndarr_feat))
y = np.concatenate((y_zeros, y_ones), axis=0)
y.shape

In [ ]:
store = pd.HDFStore('test.h5')

In [ ]:
store['test_key'] = df_flat

In [ ]:
store

In [ ]:
store.keys()

In [ ]:
df_test = store['test_key']
df_test

In [ ]:
len(df_flat.query('jet_n == 2').loc[:,'channelNumber'])

In [ ]:
type(df_flat["channelNumber"][0])

In [ ]:
type(np.int32(infos["chargino500-neutralino2L2J100"]["DSID"]))

In [ ]:
infos["chargino500-neutralino2L2J100"]["DSID"]

In [ ]:
np.int32(392302) in df_flat["channelNumber"]

In [ ]:
type(np.int32(1))

In [ ]:
df_flat["channelNumber"]

In [ ]:
392302 in set(df_flat["channelNumber"])

In [ ]:
import numpy as np
infos["chargino500-neutralino2L2J100"]["DSID"] in set(df_flat["channelNumber"])

In [ ]:
import pandas as pd

In [ ]:
pd.Series([1]*10)

In [ ]:
from infofile import infos

In [ ]:
info = {} 
for key in infos.keys(): 
        ID = infos[key]['DSID']
        info[ID] = {} 
        info[ID]['xsec'] = infos[key]['xsec'] 
        info[ID]['sumw'] = infos[key]['sumw'] 
        info[ID]['events'] = infos[key]['events']
infos

In [ ]:
10.6*0.025481788*1e3/10419.6442093

In [ ]:
import pandas as pd
df_info = pd.DataFrame(info).T
df_info

In [ ]:
import numpy as np
df_test = pd.DataFrame(data=np.arange(9).reshape((3,3)), columns=["A", "B", "C"])
df_test

In [ ]:
df_test.loc[:1,"B"] = 10
df_test

In [ ]:
df_test["index"] = df_test.index
df_test

In [ ]:
df_test = df_test.set_index("B")
df_test

In [ ]:
df_test = pd.concat([df_test, pd.DataFrame(columns=["events", "sumw", "xsec"])], axis=1)
df_test

In [ ]:
df_test.xsec.isna()

In [ ]:
df_test.index

In [ ]:
df_test.loc[10, ["events", "sumw", "xsec"]] = [1, 2, 3]
df_test

In [ ]:
df_test.insert(1, "B", df_test.index)
df_test

In [ ]:
df_test = df_test.set_index("index")
df_test

In [ ]:
df_test["B/C"] = df_test["B"]/df_test["C"]
df_test

In [ ]:
s_test = pd.Series([3, 2, 1], index=[10, 10, 7])
s_test

In [ ]:
df_test["series"] = s_test
df_test

In [ ]:
df_test.loc[5, ["events", "sumw", "xsec"]] = [1, 1, 1]
df_test

In [ ]:
i = None
1 is None

In [ ]:
j = 1
1 is j